# En Yakın K Komşu Algoritması

### Özel Kütüphane Kullanmadan KNN Algoritmasını Gerçekleştirme

İlk önce Numpy ve Pandas Kütüphanelerini çalışmamıza ekleyelim.

In [577]:
import numpy as np
import pandas as pd

Ardından Iris veri setimizi Pandas kütüphanesi yardımıyla çalışmamıza ekleyelim ve oluşturduğumuz dataframe'i sadeleştirip karıştıralım.

In [578]:
df = pd.read_csv("Iris.csv")
df.drop("Id",inplace=True,axis=1) # Model için kullanmayacağımız sütunları dataframe'den çıkaralım.
df = df.sample(frac=1) # Dataframe'i karıştıralım.
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
88,5.6,3.0,4.1,1.3,Iris-versicolor
1,4.9,3.0,1.4,0.2,Iris-setosa
135,7.7,3.0,6.1,2.3,Iris-virginica
7,5.0,3.4,1.5,0.2,Iris-setosa
87,6.3,2.3,4.4,1.3,Iris-versicolor
...,...,...,...,...,...
68,6.2,2.2,4.5,1.5,Iris-versicolor
107,7.3,2.9,6.3,1.8,Iris-virginica
48,5.3,3.7,1.5,0.2,Iris-setosa
70,5.9,3.2,4.8,1.8,Iris-versicolor


Verileri "train" ve "test" olarak ayırmak için "split_data" adında bir fonksiyon oluşturalım. Bu fonksiyon parametre olarak çiçeklerin özelliklerini içeren bir "X" numpy dizisi, çiçeklerin hangi sınıflara ait olduklarını içeren bir "y" numpy dizisi ve hangi oranda "train" ve "test" olarak ayırılacaklarına dair "train_size" adında bir yüzdelik değer alır.

In [579]:
def split_data(X, y, train_size):

    start = int(len(X)*train_size)

    X_train = X[0:start, :]
    X_test = X[start:-1, :]
    y_train = y[0:start]
    y_test = y[start:-1]

    return X_train, X_test, y_train, y_test

Şimdi sıra En Yakın K Komşu Algoritma'sını oluşturmaya geldi. Sırayla KNN sınıfı içinde oluşturduğumuz fonksiyonlara bir göz atalım;

İlk olarak "euclidean_distance ve manhattan_distance" fonksiyonlarına bakalım. Bu fonksiyonlar kendisine parametre olarak gelen birer adet test ve train verisinin karşılıklı bütün özelliklerinin arasındaki mesafeleri bulur ve bu mesafeleri toplayarak geri döndürür.

Diğer bir fonksiyonumuz ise "predict" fonksiyonu. Bu fonksiyon her bir test verisinin her bir train verisi ile arasındaki mesafeleri, karşılaştırılan train verilerinin dizinleriyle birlikte "distance" adında bir listede saklar. Ardından bu "distance" listesindeki mesafeleri yakından uzağa (küçükten büyüğe) doğru sıralar. Sonrasında ilk "k" sayısı kadarını liste içinde tutar ve diğerlerini listeden çıkarır. Artık listede sadece en yakın "k" kadar mesafe ve o mesafelerin train dizilerindeki dizinleri kalmıştır. İşte bu dizinlerin "y_train" de karşılık geldikleri sınıfları "neighbors" adlı bir listeye ekleyerek tahmin etmeye çalıştığımız çiçeğe en yakın özelliklere sahip en yakın "k" kadar çiçeğin sınıfını bulmuş oluruz. Ardından "neighbors" adlı listede hangi sınıf sayısı fazla ise sınıfını tahmin etmeye çalıştığımız çiçeği o sınıfın üyesi olarak tahmin ederiz. Ardından bu tahminlerimizi "predicted_classes" adlı bir listede toplarız ve o listeyi döndürürüz.

In [581]:
class KNN:

    def __init__(self, k, distance_type):
        self.distance_type = distance_type
        self.k = k

    def fit(self, X, y):

        assert len(X) == len(y)
        self.X_train = X
        self.y_train = y

    def euclidean_distance(self, X1, X2):

        distance = 0
        for i in range(len(X1)):
            distance += (X1[i] - X2[i]) ** 2
        return np.sqrt(distance)

    def manhattan_distance(self, X1, X2):
        distance = np.abs(np.array(X1) - np.array(X2)).sum()
        return distance

    def predict(self, X_test):

        predicted_classes = []
        
        for test_data_index in range(len(X_test)):
            
            distances = []
            for train_data_index in range(len(self.X_train)):
                if (self.distance_type == "euclidean"):
                    distance = self.euclidean_distance(self.X_train[train_data_index], X_test[test_data_index])
                elif (self.distance_type == "manhattan"):
                    distance = self.manhattan_distance(self.X_train[train_data_index], X_test[test_data_index])
                distances.append([distance, train_data_index])
            distances.sort()
            distances = distances[0:self.k]
            neighbors = []
            for distance, train_data_index in distances:
                neighbors.append(self.y_train[train_data_index])

            predicted_class = max(set(neighbors), key=neighbors.count)
            predicted_classes.append(predicted_class)
            
        return predicted_classes


Yaptığımız tahminlerin doğruluğunu ölçmek için Acuracy metriğini kullanacağız. Acuracy, sınıfları doğru tahmin edilen test verisi sayısının toplam test verisi sayısına bölünmesiyle bulunur.

In [582]:
def print_acuracy(actual, predictions):

    assert len(actual) == len(predictions)
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predictions[i]:
            correct += 1
    print("Acuracy of kNN model : ",correct / float(len(actual)))

Dataframe'de çiçeklerin özelliklerinin bulunduğu sütunları "X" adında bir değişkene, çiçeklerin sınıflarının bulunduğu sütunları ise "y" adında bir değişkene numpy dizisi olarak atayalım. Ardından bu numpy dizilerini yukarıda oluşturduğumuz "split_data" adındaki fonksiyona parametre olarak verelim.

In [589]:
X = np.array(df.iloc[:,0:-1])
y = np.array(df.iloc[:,-1:]).reshape((-1,))
X_train, X_test, y_train, y_test = split_data(X, y, train_size=0.7)

In [583]:
k=5
distance="manhattan"
knn = KNN(k,distance)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
print("Manhattan")
print_acuracy(y_test,y_pred)


Manhattan
Acuracy of kNN model :  0.9545454545454546


In [584]:
k=5
distance="euclidean"
knn = KNN(k,distance)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
print("Euclidean")
print_acuracy(y_test,y_pred)

Euclidean
Acuracy of kNN model :  0.9772727272727273


### Sckit-Learn Kütüphanesi Kullanarak KNN Algoritmasını Gerçekleştirme

In [585]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [586]:
neigh = KNeighborsClassifier(n_neighbors=5,metric="manhattan")
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
print("Manhattan")
accuracy_score(y_test, y_pred)

Manhattan


0.9555555555555556

In [587]:
neigh = KNeighborsClassifier(n_neighbors=5,metric="euclidean")
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
print("Euclidean")
accuracy_score(y_test, y_pred)

Euclidean


0.9777777777777777